See the article __[Extracting Data from Wikidata Using SPARQL and Python](https://itnext.io/extracting-data-from-wikidata-using-sparql-and-python-59e0037996f)__ by Jelle van Kerkvoorde.

The query retrieves the name, location, and founding date of all cities in the United States.

The qualifiers are:

- [`wdt:P31`](https://www.wikidata.org/wiki/Property:P31) - instance of
- [`wd:P279`](https://www.wikidata.org/wiki/Property:P279) - subclass of
- [`wd:Q515`](https://www.wikidata.org/wiki/Q515) - city
- [`wdt:P17`](https://www.wikidata.org/wiki/Property:P17) - country
- [`wd:Q30`](https://www.wikidata.org/wiki/Q30) - United States of America
- [`wdt:P625`](https://www.wikidata.org/wiki/Property:P625) - coordinate location
- [`wdt:P571`](https://www.wikidata.org/wiki/Property:P571) - inception

In [1]:
query = '''
SELECT ?city ?cityLabel ?location ?locationLabel ?founding_date
WHERE {
?city wdt:P31/wdt:P279* wd:Q515.
?city wdt:P17 wd:Q30.
?city wdt:P625 ?location.
?city wdt:P571 ?founding_date.
SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
'''

In [3]:
import python.data_extraction as DEX

data_extracter = DEX.WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()
df.head()

,city,founding_date,location,cityLabel,locationLabel
0,http://www.wikidata.org/entity/Q62,1776-06-29T00:00:00Z,Point(-122.416388888 37.7775),San Francisco,Point(-122.416388888 37.7775)
1,http://www.wikidata.org/entity/Q65,1781-09-04T00:00:00Z,Point(-118.24368 34.05223),Los Angeles,Point(-118.24368 34.05223)
2,http://www.wikidata.org/entity/Q5917,1909-02-17T00:00:00Z,Point(-117.999722222 33.692777777),Huntington Beach,Point(-117.999722222 33.692777777)
3,http://www.wikidata.org/entity/Q6586,1870-01-01T00:00:00Z,Point(-75.463333333 43.219444444),Rome,Point(-75.463333333 43.219444444)
4,http://www.wikidata.org/entity/Q16552,1769-07-16T00:00:00Z,Point(-117.1625 32.715),San Diego,Point(-117.1625 32.715)
